In [1]:
pip install -U python-jobspy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 41.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 12.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: NUMPY
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qianfan 0.3.0 requires python-dotenv<=0.21.1, but you have python-dotenv 1.0.1 which is incompatible.
langchain-openai 0.0.6 requires langchain-core<0.2,>=0.1.16, but you have langchain-core 0.2.5 which is incompatible.
langchain-community 0.0.31 requires langchain-core<0.2.0,>=0.1.37, but you have langchain-core 0.2.5 which is incompatible.
open-interpreter 0.2.4 requires fastapi<0.111.0,>=0.110.0, but

In [14]:
import csv
from jobspy import scrape_jobs
import pandas as pd
jobs = scrape_jobs(
    # site_name=["indeed", "linkedin", "zip_recruiter", "glassdoor"],
    site_name=["linkedin"],
    search_term="software engineer",
    location="Remote",
    results_wanted=1000,
    is_remote=True,
    job_type="fulltime",
    hours_old=3, # (only Linkedin/Indeed is hour specific, others round up to days old)
    country_indeed='USA',  # only needed for indeed / glassdoor
    # linkedin_fetch_description=True # get full description , direct job url , company industry and job level (seniority level) for linkedin (slower)
    # proxies=["208.195.175.46:65095", "208.195.175.45:65095", "localhost"],
    
)
print(f"Found {len(jobs)} jobs")
print(jobs.columns)
for _, job in jobs.iterrows():
    print("---")
    for column in ['id', 'site', 'job_url', 'job_url_direct', 'title', 'company',
                   'location', 'job_type', 'date_posted', 'salary_source', 'interval',
                   'min_amount', 'max_amount', 'currency', 'is_remote', 'job_level',
                   'job_function', 'company_industry', 'listing_type', 'emails'
                   , 'company_url', 'company_url_direct', 'company_addresses',
                   'company_num_employees', 'company_revenue', 'company_description',
                   'logo_photo_url', 'banner_photo_url', 'ceo_name', 'ceo_photo_url']:
        if column in job and pd.notna(job[column]):
            print(f"{column}: {job[column]}")
    print("---")
# jobs.to_csv("jobs.csv", quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False) # to_excel

2024-08-02 14:18:39,507 - JobSpy - INFO - LinkedIn search page: 1
2024-08-02 14:18:46,913 - JobSpy - INFO - LinkedIn search page: 2
2024-08-02 14:18:50,559 - JobSpy - INFO - LinkedIn search page: 3
2024-08-02 14:18:55,521 - JobSpy - INFO - LinkedIn search page: 4
2024-08-02 14:19:01,529 - JobSpy - INFO - LinkedIn search page: 5
2024-08-02 14:19:08,322 - JobSpy - INFO - LinkedIn search page: 6
2024-08-02 14:19:15,511 - JobSpy - INFO - LinkedIn search page: 7
2024-08-02 14:19:21,169 - JobSpy - INFO - LinkedIn search page: 8
2024-08-02 14:19:25,351 - JobSpy - INFO - LinkedIn search page: 9
2024-08-02 14:19:31,734 - JobSpy - INFO - LinkedIn search page: 10
2024-08-02 14:19:32,138 - JobSpy - INFO - Linkedin finished scraping


Found 78 jobs
Index(['id', 'site', 'job_url', 'job_url_direct', 'title', 'company',
       'location', 'job_type', 'date_posted', 'salary_source', 'interval',
       'min_amount', 'max_amount', 'currency', 'is_remote', 'job_level',
       'job_function', 'company_industry', 'listing_type', 'emails',
       'description', 'company_url', 'company_url_direct', 'company_addresses',
       'company_num_employees', 'company_revenue', 'company_description',
       'logo_photo_url', 'banner_photo_url', 'ceo_name', 'ceo_photo_url'],
      dtype='object')
---
id: 3990866377
site: linkedin
job_url: https://www.linkedin.com/jobs/view/3990866377
title: Frontend Developer
company: OTTO Quotes
location: 
job_level: 
company_url: https://www.linkedin.com/company/otto-quotes
---
---
id: 3989157622
site: linkedin
job_url: https://www.linkedin.com/jobs/view/3989157622
title: Software Engineer
company: Microsoft
location: Sydney, New South Wales, Australia
job_level: 
company_url: https://www.linkedin.com

In [1]:
prompt="""You are an AI assistant specialized in analyzing job postings for software engineering positions. Your task is to extract and summarize key information from the given job posting. You will be provided with a job posting, and you need to analyze it to create a concise summary for each of the following sections:

1. Team Information
2. Product Information
3. Technology Stack
4. Key Responsibilities
5. Requirements
6. Exceptional Perks or Benefits

Here is the job posting you need to analyze:

<job_posting>
{{JOB_POSTING}}
</job_posting>

Instructions for analysis:

1. Carefully read through the entire job posting.

2. For each of the six sections mentioned above, extract relevant information from the job posting. Focus on the following details for each section:

   a. Team Information:
      - Team/department name
      - Team size and structure
      - Team culture or working style
      - Cross-functional collaboration

   b. Product Information:
      - Main product or project description
      - Product impact or market position
      - Target users or industries
      - Key features or technical challenges

   c. Technology Stack:
      - Programming languages
      - Frameworks and libraries
      - Databases
      - Cloud platforms or infrastructure
      - Specialized tools or technologies

   d. Key Responsibilities:
      - Primary technical duties
      - Leadership or mentoring roles
      - Client or stakeholder interactions
      - Support or maintenance tasks
      - Project management aspects

   e. Requirements:
      - Education (degrees, fields of study)
      - Years of experience
      - Technical skills (required vs. preferred)
      - Soft skills or personal attributes
      - Industry-specific knowledge or certifications

   f. Exceptional Perks or Benefits:
      - Unusual work arrangements (e.g., 4-day work week, unlimited vacation)
      - Unique professional development opportunities
      - Exceptional financial benefits (e.g., above-market equity offers)
      - Extraordinary wellness or lifestyle perks
      - Any other benefits that are notably uncommon in the software engineering field

3. If information for a particular section or subsection is not available in the job posting, note this in your summary with a phrase such as "No information provided about [topic]."

4. Provide a concise summary for each section, focusing on the most important and relevant details. Use bullet points for clarity when appropriate.

5. Format your final output as follows:

<analysis>
<team_information>
[Your summary for Team Information]
</team_information>

<product_information>
[Your summary for Product Information]
</product_information>

<technology_stack>
[Your summary for Technology Stack]
</technology_stack>

<key_responsibilities>
[Your summary for Key Responsibilities]
</key_responsibilities>

<requirements>
[Your summary for Requirements]
</requirements>

<exceptional_perks_or_benefits>
[Your summary for Exceptional Perks or Benefits]
</exceptional_perks_or_benefits>
</analysis>

Remember to be objective and accurate in your analysis, basing your summaries solely on the information provided in the job posting. Do not make assumptions or add information that is not present in the original text."""

# Last 10 jobs

In [31]:
import boto3

# Initialize DynamoDB resource
dynamodb = boto3.resource(
    "dynamodb",
    aws_access_key_id="enter-your-access-key-id",
    aws_secret_access_key="enter-your-secret-access-key",
    region_name="us-east-1",
)
table = dynamodb.Table("jobs")

# Scan the table to fetch all jobs
all_jobs = []
last_evaluated_key = None

while True:
    if last_evaluated_key:
        response = table.scan(ExclusiveStartKey=last_evaluated_key)
    else:
        response = table.scan()
    
    all_jobs.extend(response['Items'])
    
    last_evaluated_key = response.get('LastEvaluatedKey')
    if not last_evaluated_key:
        break

print(f"Total jobs fetched: {len(all_jobs)}")

Total jobs fetched: 0


In [30]:
# Delete all items from the DynamoDB table
with table.batch_writer() as batch:
    for job in all_jobs:
        batch.delete_item(
            Key={
                'id': job['id']
            }
        )

print(f"Deleted {len(all_jobs)} jobs from the table.")


Deleted 2768 jobs from the table.


In [24]:
print(f"Fetched {len(all_jobs)} jobs:")
# Get the last 10 jobs
last_10_jobs = all_jobs[-10:]

for job in last_10_jobs:
    print(f"- {job['title']} at {job['company']}")
    print(f"  Category: {job.get('category', 'N/A')}")
    
    # Get the first 100 words of the description or description_html
    description = job.get('description_text') or job.get('description_html', '')
    words = description.split()[:100]
    truncated_description = ' '.join(words)
    
    # Print the truncated description with proper indentation
    print("  Description:")
    for line in truncated_description.split('\n'):
        print(f"    {line}")
    print()  # Add a blank line between jobs


Fetched 89 jobs:
- Network Engineer at Bayside Solutions
  Category: Not Suitable
  Description:
    Network Engineer Direct Hire Full-Time Role Salary Range: $120,000 - $140,000 per year Location: Remote PST Job Summary: As a Network Engineer, you will provide technical delivery consulting in the Infrastructure and Application Delivery lifecycles from detail design, integration, maintenance, and upgrades of hardware, network software, and standards across the enterprise as they relate to the company network. You will define and follow discreet procedures and protocols to ensure network integrity and compliance and monitor network utilization, user functions, and performance metrics as required. Duties and Responsibilities: Research, evaluate, develop, and implement network engineering designs and standards following industry best

- MSSP Partner Account Manager - Secureworks - (Remote US) at Dell Technologies
  Category: Not Suitable
  Description:
    MSSP Partner Account Manager Loc

In [25]:
for job in last_10_jobs:
    print(f"Job: {job['title']}")
    print(f"Category: {job.get('category', 'N/A')}")
    print(f"Category Explanation: {job.get('category_explanation', 'N/A')}")
    print()

Job: Network Engineer
Category: Not Suitable
Category Explanation: The job title 'Network Engineer' is unrelated to software engineering roles and does not require web or full stack development skills. It is not fully remote and likely requires less than 5 years of experience.

Job: MSSP Partner Account Manager - Secureworks - (Remote US)
Category: Not Suitable
Category Explanation: The job title is 'MSSP Partner Account Manager', which is unrelated to software engineering. It does not require web or full stack development skills and is not fully remote.

Job: Power Electronics Hardware Engineer
Category: Not Suitable
Category Explanation: Unrelated job title (Power Electronics Hardware Engineer), does not require web or full stack development skills, not fully remote, and likely requires less than 5 years of experience.

Job: Technical Lead, Software Engineer (Remote in specific locations)
Category: Strong Potential
Category Explanation: The job title 'Technical Lead, Software Enginee

In [21]:
prompt="""You are an AI assistant specialized in analyzing job postings for software engineering positions. Your task is to extract and summarize key information from the given job posting. Provide a concise summary for each section based on the available information. Each section should have a list of details.
1. Team Information

- Team/department name
- Team size and structure
- Team culture or working style
- Cross-functional collaboration

2. Product Information

- Main product or project description
- Product impact or market position
- Target users or industries
- Key features or technical challenges

3. Technology Stack

- Programming languages
- Frameworks and libraries
- Databases
- Cloud platforms or infrastructure
- Specialized tools or technologies

4. Key Responsibilities

- Primary technical duties
- Leadership or mentoring roles
- Client or stakeholder interactions
- Support or maintenance tasks
- Project management aspects

5. Requirements

- Education (degrees, fields of study)
- Years of experience
- Technical skills (required vs. preferred)
- Soft skills or personal attributes
- Industry-specific knowledge or certifications

6. Exceptional Perks or Benefits

- Unusual work arrangements (e.g., 4-day work week, unlimited vacation)
- Unique professional development opportunities
- Exceptional financial benefits (e.g., above-market equity offers)
- Extraordinary wellness or lifestyle perks
- Any other benefits that are notably uncommon in the software engineering field



Remember to keep each section's summary concise and focused on the most important points. For the Exceptional Perks or Benefits section, only include items that are truly out of the ordinary for software engineering roles."""

from pydantic import BaseModel, Field
from typing import Optional, List
import json

def litellm_completion(model, content, response_model, callback):
    import instructor
    import litellm

    litellm.success_callback = [callback]

    client = instructor.from_litellm(litellm.completion)

    return client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[{"role": "user", "content": content}],
        response_model=response_model,
    )

class JobSummary(BaseModel):
    job_id: str = Field(None, description="Job ID")
    team_information: Optional[List[str]] = Field(None, description="Summary of team information")
    product_information: Optional[List[str]] = Field(None, description="Summary of product information")
    technology_stack: Optional[List[str]] = Field(None, description="Summary of technology stack")
    key_responsibilities: Optional[List[str]] = Field(None, description="Summary of key responsibilities")
    requirements: Optional[List[str]] = Field(None, description="Summary of job requirements")
    exceptional_perks: Optional[List[str]] = Field(None, description="Summary of exceptional perks or benefits")

class JobAnalysis(BaseModel):
    job_summaries: List[JobSummary] = Field(..., description="List of job summaries")


def track_cost_callback(
    kwargs,
    completion_response,
    start_time,
    end_time,
):
    import litellm

    try:
        response_cost = kwargs.get("response_cost", 0)
        duration = end_time - start_time

        print("-" * 40)
        print("LLM Response Summary:")
        print(f"Model: {kwargs['model']}")
        print(f"Cost: ${response_cost:.6f}")
        print(f"Duration: {duration.total_seconds():.2f} seconds")

        if (
            isinstance(
                completion_response,
                litellm.ModelResponse,
            )
            and "usage" in completion_response
        ):
            usage = completion_response["usage"]
            print("Token Usage:")
            print(f"  Completion: {usage.completion_tokens}")
            print(f"  Prompt: {usage.prompt_tokens}")
            print(f"  Total: {usage.total_tokens}")
        print("-" * 40)
    except:
        print("Error occurred while printing LLM response summary")


response = litellm_completion(
    model="gpt-4o-mini",
    content=prompt + ":\n\n" + json.dumps(last_10_jobs),
    response_model=JobAnalysis,
    callback=track_cost_callback,
)


----------------------------------------
LLM Response Summary:
Model: gpt-4o-mini
Cost: $0.004829
Duration: 27.70 seconds
Token Usage:
  Completion: 1441
  Prompt: 26428
  Total: 27869
----------------------------------------


In [22]:
import json
from pprint import pprint

def truncate_description(description, word_limit=100):
    words = description.split()
    return ' '.join(words[:word_limit]) + ('...' if len(words) > word_limit else '')

job_summaries = response.job_summaries

for summary in job_summaries:
    matching_job = next((job for job in last_10_jobs if job['id'] == summary.job_id), None)
    if matching_job:
        print(f"Job ID: {summary.job_id}")
        pprint(summary.dict(), depth=2, width=120)
        print("\nJob Description (first 100 words):")
        print(truncate_description(matching_job.get('description_text', 'No description available')))
        print("\n" + "-"*80 + "\n")
    else:
        print(f"No matching job found for ID: {summary.job_id}")

Job ID: dice_4d93ba62-3374-4803-8de6-afc349811b7c
{'exceptional_perks': ['Remote work flexibility', 'Competitive salary range ($120,000 - $140,000)'],
 'job_id': 'dice_4d93ba62-3374-4803-8de6-afc349811b7c',
 'key_responsibilities': ['Research and implement network designs',
                          'Ensure compatibility with business needs',
                          'Collaborate with IT organizations',
                          'Provide technical expertise'],
 'product_information': ['Network engineering solutions for enterprise environments',
                         'Impact on network integrity and compliance'],
 'requirements': ["Bachelor's degree preferred",
                  '7+ years of network engineering experience',
                  'Senior-level certification',
                  'Basic Linux knowledge',
                  'Strong knowledge of Cisco technologies'],
 'team_information': ['Remote team structure', 'Team culture focused on technical delivery and compliance'],
 '

In [23]:
all_jobs[-15:]

[{'company_link': 'https://www.dice.com/company/summitct?companyname=Summit%20Technologies',
  'location': 'Remote',
  'timestamp': '2024-07-31 21:04:07.437462',
  'posted_date': 'Posted moments ago',
  'employment_type': 'Full-time',
  'description_html': '<div data-testid="jobDescriptionHtml"><p><strong>Summit Technologies, Inc.</strong> has a <strong>remote position</strong> available for an <strong>R&amp;D Software Engineer/Sr. Manager</strong>. In this role you will lead our client, a UK technology team by steering technical decisions and projects and ensuring the successful delivery of all technology initiatives. This position requires a strategic thinker with strong leadership and people skills, extensive experience in technical project management, and the ability to coordinate cross-functional teams to make informed technical decisions. This is a full-time temporary role lasting 9 months to 1 year. You must be available to work during UK business hours.</p><p><strong>Duties and